# Test User Class

Test user class to import the MATLAB attributions and add Markov chain development and conditioning on the Markov chain.

In [5]:
import numpy as np
import math

class User:
    """
    User: generates one user in space/time with following characteristics
        - Initial location, location at each timestep
        - User type (vehicle, pedestrian, public transport)
        - Markov chain
        - conditioning function
    """
    
    def __init__(self, boundaries, time_steps, mvmt_class, lambdas, max_speed, num_path = 1):
        """
        boundaries - x,y coordinates showing limit for where 
        time_steps - how many timesteps to simulate user movement for.
        mvmt_class - pedestrian, vehicle, or public transport (determines stochastic mvmt)
        lambdas - exponential distribution parameter for each mvmt class (list)
        numpath - number of random paths to simulate to make user markov chain
        """
        
        self.num_path = num_path
        self.time_steps = time_steps
        self.mvmt_class = mvmt_class
        self.max_speed = max_speed
        self.lmda = lambdas[mvmt_class]
        
        # Make user initial location
        init_loc = self.generate_locs(boundaries)
        
        # Draw future user location x numpath for travel
        self.all_paths = self.generate_all_paths(boundaries, init_loc, 
                                                 num_path, lambdas[mvmt_class], 
                                                 time_steps, max_speed)
        
        # User voronoi (All paths taken voronoi)
        self.user_voronoi = None
        
    """
    Init Functions
    """
    
    def generate_locs(self, boundaries):
        """
        Use uniform distribution to set server location 
        """
        
        x_min, x_max = boundaries[0,0], boundaries[0,1]
        y_min, y_max = boundaries[1,0], boundaries[1,1]
        
        locs = np.zeros(2)
        
        locs[0] = np.random.uniform(low = x_min, high = x_max, size = None)
        locs[1] = np.random.uniform(low = y_min, high = y_max, size = None)
        
        return locs

    def generate_all_paths(self, boundaries, init_loc, numpath, lmda, time_steps, max_speed):
        """
        Generate Random Movements for users starting at initial location
        """
        
        # Generate Random travel magnitude and direction from exponential distribution
        mags = np.random.exponential(1/lmda,size = (numpath, time_steps-1))
        mags[mags > max_speed] = max_speed
        angles = np.random.uniform(low = 0, high = 2 * math.pi, size = (numpath, time_steps-1))
        
        # Convert mag/angles to x,y displacements
        x_delta = np.expand_dims(np.multiply(mags, np.cos(angles)),axis=1)
        y_delta = np.expand_dims(np.multiply(mags, np.sin(angles)),axis=1)
        deltas = np.append(x_delta,y_delta,axis=1)
        
        # Add deltas to initial location while staying inside boundary
        locs = np.ones((num_path,2,time_steps)) * np.reshape(init_loc,(1,2,1))
        for t in np.arange(1,time_steps): # Offset first timestep (initloc)
            curr_locs = locs[:,:,t-1] + deltas[:,:,t-1]
            # Check if any of the boundaries are exceeded
            curr_locs = self.boundary_fix(curr_locs, boundaries)
            locs[:,:,t] = curr_locs
        
        return locs
    
    def boundary_fix(self, curr_locs,boundaries):
        """
        Shoves users to space boundary if they venture outside simulation space
        """
        
        x_min, x_max = boundaries[0,0], boundaries[0,1]
        y_min, y_max = boundaries[1,0], boundaries[1,1]
        
        x_vals = curr_locs[:,0]
        y_vals = curr_locs[:,1]
        
        x_vals[x_vals < x_min] = x_min
        x_vals[x_vals > x_max] = x_max
        y_vals[y_vals < y_min] = y_min
        y_vals[y_vals > y_max] = y_max
        
        output = np.append(np.expand_dims(x_vals,axis=1),
                           np.expand_dims(y_vals,axis=1),
                           axis=1)
        return output

    """
    Utility Functions
    """
    def generate_MC(self, servers):
        """
        Generate markov chain based on user movement patterns
        Take probabilistic conditioning on prior location to compute new location
        """
        
        # Assign closest server to each user location
        self.user_voronoi = self.find_closest_servs(servers)
        
        # Dictionary transfers between server,timestep pairs to MC nodes
        self.dict_st2node = {}
        self.dict_node2st = {}
        
        counter = 0
        for t in range(self.time_steps):
            for s in range(len(servers)):
                self.dict_st2node[(s,t)] = counter
                self.dict_node2st[counter] = (s,t)
                counter += 1
        
        # Obtain transition probabilities based on user voronoi on paths
        trans_matrix = np.zeros((counter,counter))
        
        for t in range(self.time_steps-1):
            source_servers = np.unique(self.user_voronoi[:,t])
            for s in source_servers:
                # CONTINUE HERE
        
    
    def find_closest_servs(self, servers):
        """
        Find the closest server given all user locations through time
        servers - list of server objects
        """
        
        # Make array of server locations
        server_locs = np.zeros((len(servers),2))
        for i in range(len(servers)):
            curr_svr_locs = np.expand_dims(servers[i].locs,axis=0)
            server_locs[i,:] = curr_svr_locs
        
        # Make voronoi tesselation of user locations to servers
        user_voronoi = np.zeros((self.num_path,self.time_steps))
        for n in range(self.num_path):
            for t in range(self.time_steps):
                usr_loc = np.reshape(self.all_paths[n,:,t],(1,2))
                dist_2 = np.sum((server_locs - usr_loc)**2, axis=1)
                user_voronoi[n,t] =  np.argmin(dist_2)
                
        return user_voronoi
        
        

### Make a test user case

In [2]:
boundaries =np.array([[0,100],[0,100]])
time_steps = 4
max_speed = 20
mvmt_class = 0
lamdas = [1/10]
num_path = 3

u = User(boundaries, time_steps, mvmt_class, lamdas, max_speed, num_path)
u.all_paths

array([[[24.32632335,  9.71046828,  9.77170819, 25.65834341],
        [41.10509562, 37.65397   , 37.51714827, 34.78590972]],

       [[24.32632335, 22.748264  , 34.58765571, 49.73538783],
        [41.10509562, 43.38038371, 27.26117764, 36.7102163 ]],

       [[24.32632335, 28.22547771, 18.45761725, 19.21379525],
        [41.10509562, 46.25804547, 63.710521  , 65.23018971]]])

### Generate Servers

In [3]:
from Server import Server

# Set params 
boundaries =np.array([[0,50],[0,50]])
num_resource = 2
weak_range = np.array([[2,8],[10,14]])
strong_range = np.array([[2,8],[10,14]])*2
timesteps = 3

# Number of each types of servers
num_server_l1 = 10
num_server_l2 = 3
num_server_l3 = 1

# Generate Server
servers_l1 = []
servers_l2 = []
servers_l3 = []

for i in range(num_server_l1):
    servers_l1.append(Server(boundaries,level=1,rand_locs=True,locs=None))
    
for i in range(num_server_l2):
    servers_l2.append(Server(boundaries,level=2,rand_locs=True,locs=None))
    
for i in range(num_server_l3):
    servers_l3.append(Server(boundaries,level=3,rand_locs=True,locs=None))
    
# Append all servers together
servers = servers_l1 + servers_l2 + servers_l3

### Generate User Markov Chain

In [10]:
u.user_voronoi[:,0]

array([10., 10., 10.])